In [36]:
import pickle
import torch
import pandas as pd
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification

In [37]:
#model 1
with open('models/binary_quote_model.pkl', 'rb') as f:
    model1 = pickle.load(f)
with open('models/tfidf_vectorizer.pkl', 'rb') as f:
    vectorizer = pickle.load(f)


In [38]:
#model 2
tokenizer = DistilBertTokenizerFast.from_pretrained("models/distilbert_model_final")
model = DistilBertForSequenceClassification.from_pretrained("models/distilbert_model_final")
model.eval()

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [39]:
#label mapping buat convert dari yang encoded label
df = pd.read_csv("datasets/cleanedDataSecondModel_merged.csv")
label_map = dict(df[['category_encoded', 'category']].drop_duplicates().values)

In [40]:
def full_prediction_pipeline(user_input):
    print(f"\nInput Quote: \"{user_input}\"")
    
    vec = vectorizer.transform([user_input.lower()])
    is_motivational = model1.predict(vec)[0]

    if is_motivational == 0:
        print("Model 1 → Not Motivational (x)\n")
        return

    print("Model 1 -- Motivational (v) → checking category...")

    inputs = tokenizer(user_input, return_tensors="pt", truncation=True, padding=True, max_length=128)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_label = torch.argmax(logits, dim=1).item()

    predicted_category = label_map.get(predicted_label, "Unknown")
    print(f"Model 2 -- Predicted Category: {predicted_category}\n")

In [41]:
while True:
    quote = input("Enter quote ('exit' to quit): ")
    if quote.lower() == 'exit':
        print("Goodbye!")
        break
    full_prediction_pipeline(quote)


Input Quote: "believe in yourself and everything will fall into place"
Model 1 -- Motivational (v) → checking category...
Model 2 -- Predicted Category: believe in yourself


Input Quote: "peace will always remain sacred"
Model 1 -- Motivational (v) → checking category...
Model 2 -- Predicted Category: peace


Input Quote: "you are such a sore loser"
Model 1 → Not Motivational (x)


Input Quote: "the biggest form of love is letting go"
Model 1 -- Motivational (v) → checking category...
Model 2 -- Predicted Category: love

Goodbye!
